Creation of Word Vectors

In [2]:
# Import the needed libraries
import pandas as pd
import numpy as np
import keras
import csv
from nltk.stem.porter import *
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import nltk

Using TensorFlow backend.


In [3]:
# Create a dataframe to hold our various reviews
df = pd.read_csv('/Data/data.csv')

In [4]:
# We also import some common Swahili stop words stores in a CSV file. 
# This are not all the stopwords, some may have been omitted.
stop_words_df = pd.read_csv('Data/Common Swahili Stop-words.csv')

In [5]:
stop_words = stop_words_df['StopWords'].tolist()

In [6]:
# We creat a list to hold items i.e. sentences from our imported text
text_lines = list()
lines = df['text'].values.tolist()

In [8]:
# We then use a loop to go through the dataframe and preprosses text then add these to the list created earlier
for line in lines:
    tokens = word_tokenize(line)
 
    table = str.maketrans('', '', string.punctuation) # remove punctuation
    stripped = [w.translate(table) for w in tokens] 
    words = [word for word in stripped if word.isalpha()] # remove remaining non alphabetic tokens
    words = [w for w in words if not w in stop_words] # remove common swahili stopwords
    text_lines.append(words)

In [9]:
# We then import the Gensim library and use the Word2Vec model to learn the vector represetaions 
import gensim

# Create a vocabulary to hold all words in our corpus. 
# Our vectors will however be limited to this vocab and issue may arise when the presented with a new word that wasnt in the corpus.
# The dimesnion of the vector space creted is 200. This can be adjusted and may or may not improve the accuracy of the model
vocabulary = gensim.models.Word2Vec(sentences=text_lines, size=200, window=5, workers=4, min_count=1)
words = list(vocabulary.wv.vocab)

# Just to see how mayn words we actually have.
print('Vocabulary size:  %d'  % len(words))

# Finally we can export the learnt vectors for use in other tasks e.g. Text Classification
filename = 'swahili_embeddings.txt'
vocabulary.wv.save_word2vec_format(filename, binary=False)

Vocabulary size:  542


In [10]:
# We can then test to see what words are simillar to a a given word and by what degree/score
vocabulary.wv.most_similar('pande')

[('tabora', 0.20228734612464905),
 ('ilipaswa', 0.20148324966430664),
 ('miwani', 0.18993845582008362),
 ('uliomba', 0.18693704903125763),
 ('kennedy', 0.18456421792507172),
 ('walipofika', 0.18143033981323242),
 ('uhuru', 0.17497996985912323),
 ('waliokamatwa', 0.1748640239238739),
 ('hayati', 0.1714649349451065),
 ('kwake', 0.1687362939119339)]

------

Vissualization

Credits to https://gist.github.com/BrikerMan for his source code for this generating the tow files needed for vsualization using: https://projector.tensorflow.org/

In [1]:
from gensim.models import KeyedVectors

# Load gensim word2vec
w2v_path = 'swahili_embeddings.txt'
w2v = KeyedVectors.load_word2vec_format(w2v_path)

import io

# Vector file, `\t` seperated the vectors and `\n` seperate the words
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')

# Meta data file, `\n` seperated word
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

# Write meta file and vector file
for index in range(len(w2v.index2word)):
    word = w2v.index2word[index]
    vec = w2v.vectors[index]
    out_m.write(word + "\n")
    out_v.write('\t'.join([str(x) for x in vec]) + "\n")
out_v.close()
out_m.close()